In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.svm import SVC

In [4]:
df = pd.read_csv("speeddating.csv", encoding="ISO-8859-1")



In [5]:
df.head()

,has_null,wave,gender,age,age_o,d_age,d_d_age,race,race_o,samerace,...,d_expected_num_interested_in_me,d_expected_num_matches,like,guess_prob_liked,d_like,d_guess_prob_liked,met,decision,decision_o,match
0,b'',1.0,b'female',21.0,27.0,6.0,b'[4-6]',b'Asian/Pacific Islander/Asian-American',b'European/Caucasian-American',b'0',...,b'[0-3]',b'[3-5]',7.0,6.0,b'[6-8]',b'[5-6]',0.0,b'1',b'0',b'0'
1,b'',1.0,b'female',21.0,22.0,1.0,b'[0-1]',b'Asian/Pacific Islander/Asian-American',b'European/Caucasian-American',b'0',...,b'[0-3]',b'[3-5]',7.0,5.0,b'[6-8]',b'[5-6]',1.0,b'1',b'0',b'0'
2,b'',1.0,b'female',21.0,22.0,1.0,b'[0-1]',b'Asian/Pacific Islander/Asian-American',b'Asian/Pacific Islander/Asian-American',b'1',...,b'[0-3]',b'[3-5]',7.0,NaN,b'[6-8]',b'[0-4]',1.0,b'1',b'1',b'1'
3,b'',1.0,b'female',21.0,23.0,2.0,b'[2-3]',b'Asian/Pacific Islander/Asian-American',b'European/Caucasian-American',b'0',...,b'[0-3]',b'[3-5]',7.0,6.0,b'[6-8]',b'[5-6]',0.0,b'1',b'1',b'1'
4,b'',1.0,b'female',21.0,24.0,3.0,b'[2-3]',b'Asian/Pacific Islander/Asian-American',b'Latino/Hispanic American',b'0',...,b'[0-3]',b'[3-5]',6.0,6.0,b'[6-8]',b'[5-6]',0.0,b'1',b'1',b'1'


In [1]:


# Load the data


# Preprocess the data
# Drop columns with too many missing values
df.drop(columns=["id", "idg", "condtn", "round", "position", "positin1",
                  "order", "partner", "age_o", "race_o", "pf_o_att", "pf_o_sin",
                  "pf_o_int", "pf_o_fun", "pf_o_amb", "pf_o_sha", "dec_o", "attr_o",
                  "sinc_o", "intel_o", "fun_o", "amb_o", "shar_o", "like_o",
                  "prob_o", "met_o", "sports", "tvsports", "exercise", "dining",
                  "museums", "art", "hiking", "gaming", "clubbing", "reading",
                  "tv", "theater", "movies", "concerts", "music", "shopping",
                  "yoga", "expnum", "field", "undergra", "from", "zipcode", "career",
                  "career_c", "goal", "date", "go_out", "career_c", "you_call",
                  "them_cal", "date_3", "numdat_3", "num_in_3", "dec", "field_cd",
                  "mn_sat", "tuition", "imprelig", "imprace", "frome"])
# Drop rows with missing values
df.dropna(inplace=True)

# Convert categorical variables to numerical
df = pd.get_dummies(df, columns=["gender", "race", "samerace", "field_cd", "goal",
                                 "date", "go_out"])

# Convert data to text format for vectorization
df_text = df.drop(columns=["match"]).astype(str).apply(lambda x: ' '.join(x), axis=1)

# Vectorize the data using TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df_text).toarray()
y = df["match"].to_numpy()

# Compute the kernel matrix
K = cosine_similarity(X)

# Train and evaluate the model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
K_train = K[X_train[:, 0].astype(int), :][:, X_train[:, 0].astype(int)]
K_test = K[X_test[:, 0].astype(int), :][:, X_train[:, 0].astype(int)]

svm = SVC(kernel='precomputed')
svm.fit(K_train, y_train)
scores = cross_val_score(svm, K_train, y_train, cv=5)

print('Cross-validation scores:', scores.mean())
print('Test accuracy:', svm.score(K_test, y_test))

KeyError: "['id', 'idg', 'condtn', 'round', 'position', 'positin1', 'order', 'partner', 'pf_o_att', 'pf_o_sin', 'pf_o_int', 'pf_o_fun', 'pf_o_amb', 'pf_o_sha', 'dec_o', 'attr_o', 'sinc_o', 'intel_o', 'fun_o', 'amb_o', 'shar_o', 'like_o', 'prob_o', 'met_o', 'expnum', 'undergra', 'from', 'zipcode', 'career', 'career_c', 'goal', 'date', 'go_out', 'career_c', 'you_call', 'them_cal', 'date_3', 'numdat_3', 'num_in_3', 'dec', 'field_cd', 'mn_sat', 'tuition', 'imprelig', 'imprace', 'frome'] not found in axis"